<a href="https://colab.research.google.com/github/SherifNabil-Eng/AzureCognitivePoCs/blob/main/TestFormRecGeneralAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 !pip install azure-ai-formrecognizer==3.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.4/228.4 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 KB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.5/174.5 KB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 KB 1.9 MB/s eta 0:00:00


In [ ]:
# import libraries
from google.colab import drive
drive.mount('/content/gdrive')
#drive.mount('/content/drive', force_remount=force_remount)

import os
import sys
import numpy as np
import pandas as pd
from datetime import date

from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential
from azure.core.exceptions import HttpResponseError

os.chdir("/content/gdrive/My Drive/Colab Notebooks/Tests")


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
def format_bounding_region(bounding_regions):
    if not bounding_regions:
        return "N/A"
    return ", ".join("Page #{}: {}".format(region.page_number, format_polygon(region.polygon)) for region in bounding_regions)

def format_polygon(polygon):
    if not polygon:
        return "N/A"
    return ", ".join(["[{}, {}]".format(p.x, p.y) for p in polygon])


In [ ]:
def analyze_doc(document):

  endpoint = ""
  key = ""
  lines=[]

  document_analysis_client = DocumentAnalysisClient(
      endpoint=endpoint, credential=AzureKeyCredential(key)
      )
  
  print ("------{} document to be analyzed ".format(document))
  
  with open(document, "rb") as f:
        poller = document_analysis_client.begin_analyze_document(
            "prebuilt-document", document=f
        )
  result = poller.result()

  for style in result.styles:
        if style.is_handwritten:
            print("Document contains handwritten content: ")
            print(",".join([result.content[span.offset:span.offset + span.length] for span in style.spans]))

  print("----Key-value pairs found in document----")
  
  for kv_pair in result.key_value_pairs:
    if kv_pair.key:
      print("Key '{}' found within '{}' bounding regions".format(
          kv_pair.key.content,format_bounding_region(kv_pair.key.bounding_regions),
          )
      )
    if kv_pair.value:
      print( "Value '{}' found within '{}' bounding regions\n".format(
          kv_pair.value.content,format_bounding_region(kv_pair.value.bounding_regions),
          )
      )

  for page in result.pages:
        print("----Analyzing document from page #{}----".format(page.page_number))
        print(
            "Page has width: {} and height: {}, measured with unit: {}".format(
                page.width, page.height, page.unit
            )
        )

        for line_idx, line in enumerate(page.lines):
            words = line.get_words()
            print(
                "...Line # {} has {} words and text '{}' within bounding polygon '{}'".format(
                    line_idx,
                    len(words),
                    line.content,
                    format_polygon(line.polygon),
                )
            )
            lines.append(line.content)

            for word in words:
                print(
                    "......Word '{}' has a confidence of {}".format(
                        word.content, word.confidence
                    )
                )

        for selection_mark in page.selection_marks:
            print(
                "...Selection mark is '{}' within bounding polygon '{}' and has a confidence of {}".format(
                    selection_mark.state,
                    format_polygon(selection_mark.polygon),
                    selection_mark.confidence,
                )
            )
    

  f.close()
  
  return lines

In [ ]:
def make_file_path(document_file_name):
  
  currroot=os.getcwd()
  print("File location using os.getcwd():", os.getcwd())
  document_path_temp = os.path.join(currroot, document_file_name)

  return document_path_temp

In [ ]:
if __name__ == "__main__":

  documents =["FormRecImgs/bday_Farah.jpg","FormRecImgs/Marriage_sherif_yasmine.jpg"]
  
  document_lines=[]
 

  try:
    for i in range (0,len(documents)):
      document_path= make_file_path(documents[i])
      #document_content = analyze_doc(document_path)
      document_lines.append(analyze_doc(document_path))
  except HttpResponseError as error:
    
    # Examples of how to check an HttpResponseError
    # Check by error code:
    if error.error is not None:

      if error.error.code == "InvalidRequest":

        print(f"Received an invalid request error: {error.error}")
        sys.exit(1)
      if error.error.code == "InvalidImage":

        print(f"Received an invalid image error: {error.error}")
        sys.exit(1)
        # If the inner error is None and then it is possible to check the message to get more information:
    filter_msg = ["Generic error", "Timeout", "Invalid request", "InvalidImage"]
    if any(example_error.casefold() in error.message.casefold() for example_error in filter_msg):
      print(f"Uh-oh! Something unexpected happened: {error}")
      sys.exit(1)
        # Print the full error content:
    print(f"Full HttpResponseError: {error}")

File location using os.getcwd(): /content/gdrive/MyDrive/Colab Notebooks/Tests
------/content/gdrive/MyDrive/Colab Notebooks/Tests/FormRecImgs/bday_Farah.jpg document to be analyzed 
----Key-value pairs found in document----
----Analyzing document from page #1----
Page has width: 697.0 and height: 1024.0, measured with unit: pixel
...Line # 0 has 5 words and text 'أهلة القاهرة سحا مدنى الخزانة' within bounding polygon '[78.0, 150.0], [193.0, 168.0], [188.0, 202.0], [75.0, 190.0]'
......Word 'أهلة' has a confidence of 0.207
......Word 'القاهرة' has a confidence of 0.604
......Word 'سحا' has a confidence of 0.055
......Word 'مدنى' has a confidence of 0.375
......Word 'الخزانة' has a confidence of 0.115
...Line # 1 has 3 words and text 'صورة قيد الميلاد:٢' within bounding polygon '[216.0, 172.0], [418.0, 166.0], [419.0, 207.0], [216.0, 210.0]'
......Word 'صورة' has a confidence of 0.48
......Word 'قيد' has a confidence of 0.826
......Word 'الميلاد:٢' has a confidence of 0.106
...Line # 2 

In [ ]:
print (document_lines[0])

['أهلة القاهرة سحا مدنى الخزانة', 'صورة قيد الميلاد:٢', 'حمد هُورَة مَصِر العَربِيمُ', 'وَزَارِقَ الْكَ أَخْيَّةُ', 'قَطَاعُ الْأَحَوَانِ المَاسَةُ به', 'www.cso.gov.eg', '٣', 'القرى: ٠٠٤٤٧ ٨٨ ٩٠٣٢٢', 'بَيَانَائ المولود', 'الديانة : مسيحية', 'النوع: انشى', 'فرح', 'الجنسية : مصر', 'امريكا', 'محل الميلاد : الخارج', 'تاريخ الميلاد : اثنين وعشرون من مارس', 'عام الفان وتسعه عشر', 'الا', 'بَيَانَا رَ الأبْ', 'شريف نبيل جرجس اسحق يوسف', 'I', 'الرقم القومى: ٢٧٦١٠٠٦٠١٠٩٢٥١', 'الديانة : مسيحى', 'الجنسية : مصر', 'بَيَانات الأمْ', '6 4', 'ياسمين عماد رمزى ويصا', 'الرقم القومى: ٢٨٤٠٣٢١٠١٠٦٦٤٥', 'الديانة : مسيحية', 'الجنسية : مصر', 'رقم القيد ٠١١٢٢٣٨٧', 'ت. القيد :٠٧ /٠٥/ ٢٠١٩', 'ت.أصد ار ٤ ١٢ / ٠٩/ ٢٠٢٠١٩', '. صحة : المركز الرئيسى', 'م.', 'س . مدني: المركز الرئيسى', 'س ٠ ١ صد آراء سجل البرهه', 'السيـ', 'توقسين', 'ضابطا شركة', '٢٥٥٤٣٦١١٢١', 'رقم مسلسل', 'رائد / مصطفى محمود عدلان', 'مفتش بيلات', 'تم من شرق', '10', 'تأكد من وجولا طابع الأمومة والطفولة فئة ١ جنيه والعلا']


In [ ]:
for i in range (0,len(documents)):
  index_slash=documents[i].find ("/")
  index_dot=documents[i].find (".")

  file_sub_name = documents[i][index_slash+1:index_dot]

  current_date = today = str (date.today())

  extracted_file_name = file_sub_name+"FormRecGeneral"+current_date+".txt"

  file_extracted_handle = open(extracted_file_name,"a")

  for j in range (0,len(document_lines[i])):
    file_extracted_handle.writelines(document_lines[i][j]+"\n")
  


file_extracted_handle.close()